In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, LeaveOneOut
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.metrics import  make_scorer, accuracy_score, f1_score, confusion_matrix, roc_auc_score, silhouette_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
import joblib

In [2]:
!where python

c:\Users\polyx\Desktop\Pattern-Recognition-Gr7\venv\Scripts\python.exe
C:\Users\polyx\AppData\Local\Programs\Python\Python310\python.exe
C:\Users\polyx\AppData\Local\Microsoft\WindowsApps\python.exe


In [35]:
import sklearn
print(np.__version__)

2.1.2


In [3]:
with open('all_cv_grid_search_results.pkl', 'rb') as file:
    grid_search_results = pickle.load(file)


print(grid_search_results['KFold_2']['grid'].cv_results_)

{'mean_fit_time': array([62.81941545, 62.63066113, 61.84940863,  0.43674004,  0.4470948 ]), 'std_fit_time': array([4.00435567e-01, 1.43224835e-01, 2.48682737e-01, 2.68340111e-04,
       1.90166235e-02]), 'mean_score_time': array([0.08143139, 0.08802009, 0.08623457, 0.10415721, 0.62784946]), 'std_score_time': array([2.28476524e-03, 2.30789185e-04, 2.95639038e-05, 9.67979431e-05,
       6.28244877e-03]), 'param_classifier': masked_array(data=[RandomForestClassifier(random_state=12),
                   KNeighborsClassifier(), KNeighborsClassifier(),
                   SVC(random_state=12), SVC(random_state=12)],
             mask=[False, False, False, False, False],
       fill_value=np.str_('?'),
            dtype=object), 'param_feature_selection': masked_array(data=[SelectKBest(score_func=<function mutual_info_classif at 0x00000289BB349090>),
                   SelectKBest(score_func=<function mutual_info_classif at 0x00000289BB349090>),
                   SelectKBest(score_func=<funct

In [8]:
def create_ranked_lists(grid_search_results):

    cv_results = grid_search_results['KFold_2']['grid'].cv_results_
    
    df = pd.DataFrame({
        'mean_fit_time': cv_results['mean_fit_time'],
        'mean_score_time': cv_results['mean_score_time'],
        'mean_test_f1': cv_results['mean_test_f1'],
        'params': cv_results['params']  # Access to model configurations
    })

    # Rank models based on mean_score_time
    df_ranked_by_score_time = df.sort_values(by='mean_score_time', ascending=True).reset_index(drop=True)

    # Rank models based on mean_fit_time
    df_ranked_by_fit_time = df.sort_values(by='mean_fit_time', ascending=True).reset_index(drop=True)

    return df_ranked_by_score_time, df_ranked_by_fit_time

In [15]:
ranked_by_score_time[['params'][0]][0]

{'classifier': RandomForestClassifier(random_state=12),
 'feature_selection': SelectKBest(score_func=<function mutual_info_classif at 0x00000289BB349090>),
 'feature_selection__k': 500}

In [9]:
ranked_by_score_time, ranked_by_fit_time = create_ranked_lists(grid_search_results)

print("Ranked by Mean Score Time:")
print(ranked_by_score_time[['mean_score_time', 'mean_test_f1', 'params']])
print("\nRanked by Mean Fit Time:")
print(ranked_by_fit_time[['mean_fit_time', 'mean_test_f1', 'params']])

Ranked by Mean Score Time:
   mean_score_time  mean_test_f1  \
0         0.081431      0.995304   
1         0.086235      0.989112   
2         0.088020      0.990647   
3         0.104157      0.795700   
4         0.627849      0.995305   

                                              params  
0  {'classifier': RandomForestClassifier(random_s...  
1  {'classifier': KNeighborsClassifier(), 'classi...  
2  {'classifier': KNeighborsClassifier(), 'classi...  
3  {'classifier': SVC(random_state=12), 'feature_...  
4               {'classifier': SVC(random_state=12)}  

Ranked by Mean Fit Time:
   mean_fit_time  mean_test_f1  \
0       0.436740      0.795700   
1       0.447095      0.995305   
2      61.849409      0.989112   
3      62.630661      0.990647   
4      62.819415      0.995304   

                                              params  
0  {'classifier': SVC(random_state=12), 'feature_...  
1               {'classifier': SVC(random_state=12)}  
2  {'classifier': KNeighborsCl